Reference: 
    1. "Tensor Decompositions and Applications", Tamara Kolda, Brett W. Bader, 
    2. Multi-way Analysis by Age Smilde, R Bros 
Author: Suhan Shetty <suhan.n.shetty@idiap.ch>

Date: 29th July 2019

This is an implementation of common tools for 3-way array
    - Generate a 3-way tensor data {Input(X),Output(Y)}
    - Tensor Operations
    - Tensor Decomposition Techniques
    - Tensor-Tensor Regression
    - Array-normal distribution
   

In [12]:
import numpy as np
from numpy.linalg import multi_dot as mat_mul
from numpy.random import random_sample 

In [13]:


"""   
A note about convention used:
numpy array index starts from 0. However, we use for the mode numbering startion from 1.
So, axis 0 in a numpy array corresponds to mode 1.
"""

class array3:
    
#-----------------------------------------------------------------------------------------------------------------
    # Genrate a random 3-way array
    def random_array3(self,tensor_shape):
        #tensor_shape.reverse()
        t_shape = [tensor_shape[2],tensor_shape[0],tensor_shape[1]]
        X = 0
        R = np.random.randint(10)+10 # Randomize R, number of true components
        for p in range(R): # Add some true components to the tensor
            a = random_sample((tensor_shape[0],))
            b = random_sample((tensor_shape[1],))
            c = random_sample((tensor_shape[2],))
            X = X + self.outer_product(a,b,c) 
        X = X + np.random.random(t_shape) # Add some noise over the true component
        return X


# Shape of the tensor --------------------------------------------------------------------------------------------

    def shape(self,X): #
        return (X.shape[1],X.shape[2],X.shape[0])

# Compute kronecker product of the set of matrices ---------------------------------------------------------------
    
    def kron(self,A): # Here, A=[A1,A2,A3,..,AN]
        N = len(A)
        kp = np.kron(A[0],A[1])
        k = 2
        while k<N:
            kp = np.kron(kp,A[k])
            k=k+1
        return kp
    
# Unfold the tensor along the given mode --------------------------------------------------------------------------
   
    def unfold(self,X,n):
        if n==2:
            X = np.swapaxes(X,1,2) 
        elif n==3:
            X = np.swapaxes(X,0,2)
            X = np.swapaxes(X,1,2)
        depth = X.shape[0]
        Xn = X[0,:,:] 
        for k in range(depth-1):
            tmp = X[k+1,:,:]
            Xn = np.concatenate((Xn,tmp),axis=1)
        return Xn
    
# Unfold the tensor along the given mode for 4-way array ---------------------------------------------------------
   
    def unfold4(self,X,n):
        if n==1:
            m = X.shape[2]
            X = np.swapaxes(X,0,2)
            X = np.swapaxes(X,1,2)
        elif n==2:
            m = X.shape[3]
            X = np.swapaxes(X,0,3)
            X = np.swapaxes(X,1,3)
            X = np.swapaxes(X,2,3)
        elif n==3:
            
            m = X.shape[1]
            X = np.swapaxes(X,0,1)
            
        elif n==4:
            m = X.shape[0]

       
        Xn = X.flatten('C')
        l = len(Xn)
        Xn = np.reshape(Xn,[m,int(l/m)])
        return Xn
    
# mode-n product of the tensor with a matrix-----------------------------------------------------------------------
   
    # For 3-way tensor
    def mode_n_product(self,X,M,n): # T_out = T_in * M
        if n==2:
            X = np.einsum('ijl,kl',X,M)
        elif n==1:
            X = np.einsum('ilk,jl',X,M)
        elif n==3:
            X = np.einsum('ljk,il',X,M)
        return X
    # For 4-way tensor
    def mode_n_product4(self,X,M,n): # T_out = T_in * M
        if n==2:
            X = np.einsum('hijl,kl',X,M)
        elif n==1:
            X = np.einsum('hilk,jl',X,M)
        elif n==3:
            X = np.einsum('hljk,il',X,M)
        elif n==4:
            X = np.einsum('lijk,hl',X,M)
        return X



# Compute the Frobenius norm -------------------------------------------------------------------------------------
   
    def norm(self,X):
        return np.linalg.norm(X.flatten())

# Vectorize a tensor----------------------------------------------------------------------------------------------
   
    def vectorize(self,X):
        return X.flatten()
            
# Khatri-Rao product of two matrices------------------------------------------------------------------------------
    
    def khatri_rao(self,M):
        X = M[0]
        Y = M[1]
        assert X.shape[1]==Y.shape[1],"Number of columns must match"
        I = X.shape[0]*Y.shape[0]
        J = X.shape[1]
        Z = np.empty([I,J])      
        for k in range(J):
            Z[:,k] = np.kron(X[:,k],Y[:,k])
        return Z

# Hadamard Product -----------------------------------------------------------------------------------------------

    def hadamard(self,X,Y):
        assert X.shape==Y.shape,"Dimensions must match"
        return X*Y

# Outer Product ---------------------------------------------------------------------------------------------------

    def outer_product(self,a,b,c): #outer product of three vectors
        assert a.ndim == 1 and b.ndim==1 and c.ndim==1, "Input has to be 1d arrays"
        return np.einsum('i,k,j',c,b,a) # or np.einsum('i,j,k',a,b,c).swapaxes(0,2)


# CP/CANDECOMP/PARAFAC decompostion of a tensor ------------------------------------------------------------------
    # Uses ALS technique to decompose the tensor into R rank-1 tensors
    def cp(self,X,R):
        I = self.shape(X)
        r = min([I[0]*I[1], I[1]*I[2], I[0]*I[2]])
        assert R<r, "Reduce R. Its too large comapred to the dimension of the data"
             
        # Initialization of factor matrices
        A = [np.random.random((I[k],R)) for k in range(3)]
        #print("Shape of A: ", A[0].shape, A[1].shape, A[2].shape)
        # Unfold X along every mode
        Xn = [self.unfold(X,k+1) for k in range(3)]
        # Alternative way to initialize A is to take first R left singular vectors from unfolded tensors
        #for k in range(3):
            #U,S,V = np.linalg.svd(Xn[k])
            #A[k] = U[:,0:R]
        # Iterative step of ALS
        for reps in range(10000):
            tmp = A[:]
            for k in range(3):# iterate for each mode
                idx = [0,1,2]
                idx.remove(k)
                #print("Shape 0 and 1: ",A[idx[0]].shape, A[idx[1]].shape )
                Z = self.khatri_rao([A[idx[1]],A[idx[0]]])
                W = self.hadamard(mat_mul([A[idx[0]].T,A[idx[0]]]),mat_mul([A[idx[1]].T,A[idx[1]]]))
                Winv = np.linalg.pinv(W,rcond=0.001)
                A[k] = mat_mul([Xn[k],Z,Winv])         
            if (reps+1)%100==0:
                err_ = sum([np.linalg.norm(A[p]-tmp[p]) for p in range(3)])
                if err_<0.001:
                    print("The algorithm has converged. Number of iterations in ALS: ",reps)
                    break
                else:
                    tmp = A[:]  
        return A # Return A = [[a1,a2,..,aR], [b1,b2,...,bR],[c1,c2,...,cR]]
    
# Tucker product--------------------------------------------------------------------------------------------------

    # Tucker product of a tensor with a list of factor matrices
    def tucker_product(self,X, M, I): # M = [Mp,Mq,Mr,..,Mn] , I: Index for the corresponding mode I = [p,q,..n]
        if len(X.shape)==3:
            for i in range(len(I)):
                X = self.mode_n_product(X,M[i],I[i])
        elif len(X.shape)==4:
            for i in range(len(I)):
                X = self.mode_n_product4(X,M[i],I[i])
            
        return X

# Tucker Matrix Product ------------------------------------------------------------------------------------------
    
    def tucker_matrix_product(self,X, M_,n): # M = [M1,M2,M3,..,MN] 
        M = M_[:]
        Mn = M[n-1]
        M.pop(n-1)
        M.reverse()
        M = [M[k].T for k in range(len(M))]
        Z = self.kron(M)
        Xn = self.unfold(X,n)
        Yn = mat_mul([Mn,Xn, Z])
        return Yn # Returns the unfolded tensor along mode n after taking tucker product    
      
# Tucker Decompostion---------------------------------------------------------------------------------------------

    # Technique: HOOI(Higher-order orthogonal iteration)    
    def tucker3(self,X,R):# Find the decompostion X = G x {A1,A2,A3}
            assert len(R)==3, "Input list should have exactly three elements"
            I = self.shape(X)
            A = [np.random.random((I[k],R[k])) for k in range(3)] #Initialize the factors 
            Xn = [self.unfold(X,n+1) for n in range(3)]
            # Iterate till convergence or max_step
            max_step = 1000
            for reps in range(max_step):
                tmp = A[:]
                for k in range(3): # for each mode
                    idx = [0,1,2]
                    idx.remove(k)
                    Z = self.kron([A[idx[1]], A[idx[0]]])
                    Yk = mat_mul([Xn[k],Z])
                    U,S,V = np.linalg.svd(Yk)
                    A[k] = (U[:,0:R[k]])# Take the R[k] leading left singular vectors of Yk
                if (reps+1)%10==0: # Check for convergence
                    err_ = sum([np.linalg.norm(A[p].flatten()-tmp[p].flatten()) for p in range(3)])
                    if err_<= 0.001:
                        print("The algorithm has converged. Number of iterations: ",reps)
                        break
                    else:
                        tmp = A[:]   
            At = [A[k].T for k in range(len(A))]
            G = self.tucker_product(X,At,[1,2,3])
            return (A,G)

# Partial Tucker--------------------------------------------------------------------------------------------------------#
   
    # Find the decompostion X = G x {A1,A2,A3} given one of the factors
    def partial_tucker3(self,X,R,A_,mode):
            assert len(R)==3, "Input list should have exactly three elements"
            I = self.shape(X)
            A = [np.random.random((I[k],R[k])) for k in range(3)] #Initialize the factors 
            A[mode-1] = A_
            Xn = [self.unfold(X,n+1) for n in range(3)]
            # Iterate till convergence or max_step
            max_step = 1000
            id_ = [0,1,2]
            id_.remove(mode-1)
            for reps in range(max_step):
                tmp = A[:]
                for k in id_: # for each mode
                    idx = [0,1,2]
                    idx.remove(k)
                    Z = self.kron([A[idx[1]], A[idx[0]]])
                    Yk = mat_mul([Xn[k],Z])
                    U,S,V = np.linalg.svd(Yk)
                    A[k] = (U[:,0:R[k]])# Take the R[k] leading left singular vectors of Yk

                if (reps+1)%10==0: # Check for convergence
                    err_ = sum([np.linalg.norm(A[p].flatten()-tmp[p].flatten()) for p in range(3)])
                    if err_<= 0.001:
                        #print("The algorithm has converged. Number of iterations: ",reps)
                        break
                    else:
                        tmp = A[:]   
                        
            At = [A[k].T for k in range(len(A))]
            G = self.tucker_product(X,At,[1,2,3])
            #A = [At[k].T for k in range(3)]
            return (A,G)
            
# Covariate Tensor-Tensor Regression------------------------------------------------------------------------------
   
    # Assumption: The samples are stacked along the third mode of the tensor 
    def cov_regression(self,X_Data,Y_Data, alpha, R_X, R_Y):
        # alpha is between 0 and 1. alpha is approximately (input fitting)/(output correlation) coefficient
        # R_X and R_Y are dimensions of tucker factors for X_Data and Y_Data respectively
        I_X = self.shape(X_Data)
        I_Y = self.shape(Y_Data)
        assert I_X[2]==I_Y[2] and R_X[2]==R_Y[2], "The number of samples must equal the dimension in the third mode"
        C = np.random.random((I_X[2],R_X[2])) # C_X = C_Y
        C_old = np.copy(C)
        for reps in range(1000):
            (Ax,Gx_)= self.partial_tucker3(X_Data,R_X, C, mode=3)
            (Ay,Gy_)= self.partial_tucker3(Y_Data,R_Y, C, mode=3)
            Gx = self.unfold(Gx_,3)
            Gy = self.unfold(Gy_,3)
            X_flat = self.unfold(X_Data,3)
            Y_flat = self.unfold(Y_Data,3)
            Px = mat_mul([Gx,self.kron([Ax[1].T,Ax[0].T])])
            Py = mat_mul([Gy,self.kron([Ay[1].T,Ay[0].T])])
            X_pinv = np.linalg.pinv(X_flat,rcond=0.001)
            Z = np.sqrt(alpha)*mat_mul([X_flat,Px.T])+np.sqrt(1-alpha)*mat_mul([Y_flat,Py.T])
            #Alternatively, U = np.sqrt(alpha)*mat_mul([Px,Px.T])+np.sqrt(1-alpha)*mat_mul([Py,Py.T])
            U = np.sqrt(alpha)*mat_mul([Gx,Gx.T])+np.sqrt(1-alpha)*mat_mul([Gy,Gy.T])
            X_pinv = np.linalg.pinv(X_flat)
            U_pinv = np.linalg.pinv(U)
            W = mat_mul([X_pinv,Z,U_pinv])
            C = mat_mul([X_flat,W])
            if (reps+1)%500==0:
                if np.linalg.norm(C-C_old)<0.1:
                    print("The algorithm has converged. Number of iterations: ",reps)
                    break
        X_fit = mat_mul([C,Px])
        Error = np.linalg.norm(X_fit-X_flat)
        print("Error in input data fit: ", Error, "and norm of the input data: ",np.linalg.norm(X_flat) )     
        return (Ax,Gx,Ay,Gy,W,Py)

# Compute Matrix root--------------------------------------------------------------------------------------
    # Compute A's from Cov: Cov = U*S*Vh, A = U*Sqrt(S)
    def mat_root(self,Cov):
        U,s,Vh = np.linalg.svd(Cov)
        s_r = np.sqrt(s)
        if s_r.any()==0:
            print("Error: Cov is not conditioned well")
#         s_inv_r = 1/s_r
#         s_inv_r[s_inv_r>1000000] = 0
#         s_inv_r[s_inv_r<1000000] = 0
        S_r = np.diag(s_r)
        S_inv_r = np.linalg.inv(S_r)#np.diag(s_inv_r) 
        A = mat_mul([U,S_r])
        A_inv = mat_mul([S_inv_r, U.T])
        return (A, A_inv)
    
# Array-Normal Distribution--------------------------------------------------------------------------------------
  
    # Separable Covariance estimation for 3-way array data using MLE
    def anormal(self, X, coef = 0.5, constraint = False): # X = {X1, X2,...,Xn}
        I = self.shape(X[0])
        N = len(X)
        # Compute the mean:
        M = 0
        for X_ in X:
            M = M + X_
        M = M/N
        shape_ = list(X[0].shape)
        shape_.insert(0,N)
        M_ext = np.empty(shape_)
        X_ext = np.empty(shape_)
        # Extended array mean and array
        for i in range(N):
            M_ext[i,:,:,:] = M 
            X_ext[i,:,:,:] = X[i]
        # Residual:
        E = X_ext - M_ext
        # X_i ~ M + Z x {Cov1, Cov2, Cov3}, and CovK = Ak*Ak'
        # Intialize the covariance matrices (mode-1, mode-2,mode-3)
        # mode-1 covariance = column cov, mode-2 cov= row covariance, mode-3 is pipe cov
        Cov = [np.random.rand(I[i],I[i]) for i in range(3)]
        Cov[0] = mat_mul([Cov[0],Cov[0].T])
       # print("Cov[0]: ", Cov[0])
        Cov[1] = mat_mul([Cov[1],Cov[1].T])
        Cov[2] = mat_mul([Cov[2],Cov[2].T])
        if constraint==True:
            # Give auto-regressive structure to the longituduinal covariance matrix
            corr_ = [coef**j for j in range(I[2])]
            Cov[2][0,:] = np.array(corr_)
            for j in range(I[2]-1):
                el = corr_.pop()
                corr_.insert(0,coef**(j+1))
                Cov[2][j+1,:] = np.array(corr_)
        A = [None]*3
        A_inv = [None]*3
        for j in range(3):
            (A[j],A_inv[j]) = self.mat_root(Cov[j])
        A_inv_ext = A_inv[:]
        A_inv_ext.append(np.identity(N))
        
        for reps in range(1000):
            tmp = Cov[:]
            #pdb.set_trace()
            for k in range(2): #iterate over each mode
                idx = [0,1]
                idx.pop(k)
                for j in idx:
                    (A[j], A_inv_ext[j]) = self.mat_root(Cov[j])
                A_inv_ext[k] = np.identity(I[k])
                E_ = self.tucker_product(E,A_inv_ext,[1,2,3,4])
                E_k = self.unfold4(E_,k+1)
                S = mat_mul([E_k,E_k.T])
                nk = N*np.prod(I)/I[k]
                Cov[k] = S/nk
                
            if (reps+1)%10==0:
                err_ = sum([np.linalg.norm(Cov[p]-tmp[p]) for p in range(3)])
                if err_ < 0.001:
                    print("MLE converged in ", reps, " steps")
                    break
                else:
                    tmp = Cov[:]
        return (M,Cov,A)

#  Array-normal Conditioning--------------------------------------------------------------------------------------
   
    # 3-way array-normal conditioning
    def anormal_condition(self,M,Cov,Ia,X_a, slice_):
        # mode: along which mode unfolding each column is partially known
        # Ia: Index of the know data along the given mode
        # X_a: data slice at index Ia
        # mode = 1,2,3 => incomplete info along each row, columns, pipe respectively
        Ix = M.shape
        if slice_ == 1:# incomplete info along each row when unfoded along mode 2 OR slice given along mode 1
            mode = 2
            Ib = set(range(Ix[2]))
            Ib = list(Ib - set(Ia)) #index of unknown row elements
            Cov_mode = Cov[mode-1] # row covariance i.e. covariance along mode 2 unfolding
            M_b = M[:,:,Ib] 
            M_a = M[:,:,Ia]
        elif slice_ == 2:# incomplete info along each column when unfolded along mode 1 OR slice given along mode 2
            mode =  1
            Ib = set(range(Ix[1]))# index of unknown columns
            Ib = list(Ib - set(Ia))
            Cov_mode = Cov[mode-1] # row covariance  
            M_b = M[:,Ib,:]
            M_a = M[:,Ia,:]
        elif slice_==3:# incomplete info along each pipe
            mode = 3
            Ib = set(range(Ix[0]))# index of unknown columns
            Ib = list(Ib - set(Ia))
            Cov_mode = Cov[mode-1] # row covariance  
            M_b = M[Ib,:, :]
            M_a = M[Ia,:,:]
        else:
            raise ValueError('Invalid mode passed')
        Cov_ba = Cov_mode[np.ix_(Ib,Ia)]
        Cov_aa = Cov_mode[np.ix_(Ia,Ia)]
        Cov_bb = Cov_mode[np.ix_(Ib,Ib)]
        inv_Cov_aa = np.linalg.inv(Cov_aa)
        update_coef = mat_mul([Cov_ba,inv_Cov_aa]) 
        Cov_ = Cov[:] 
        Cov_[mode-1] = Cov_bb - mat_mul([update_coef,Cov_ba.T])
        M_ = M_b + self.mode_n_product((X_a-M_a),update_coef,mode)  
        return (M_,Cov)
        
# Sampling from array-normal distribution---------------------------------------------------------------------------
    
    def anormal_sampling(self,M,Cov):
        A = [None]*3 #Cov[:]
        # Compute the matrix-square-root (Cov = A*A') of covariance matrices
        for j in range(3):             
            A[j],_ = self.mat_root(Cov[j])
        Z = np.random.randn(*M.shape)
        for j in range(3):
            X_ = M + self.tucker_product(Z,A,[1,2,3])
        return X_
                


In [14]:
# Specify the tensor
from tensor_tools import array3
ar3 = array3()
tensor_shape = [8,3,2] # [mode_1, mode_2 mode_3]: [rows, columns, tubes]


In [15]:
# PARAFAC/CP

# Data
X = ar3.random_array3(tensor_shape)

R = 5 #Number of rank-1 components in the CP

#(A,G) = ar3.cp(X,R) #CP decompostion A = [[a1,a2,..,aR], [b1,b2,...,bR],[c1,c2,...,cR]]
A = ar3.cp(X,R)

# D_ = [np.diag(G[i].flat) for i in range(3)]
# D = mat_mul([D_[0],D_[1],D_[2]])#np.identity(R)#

X_apprx1 = mat_mul([A[0],(ar3.khatri_rao([A[2],A[1]])).T])

Error1 = ar3.unfold(X,1) - X_apprx1

print("Norm of the Residual: ", np.linalg.norm(Error1))

print("Norm of the Data: ", np.linalg.norm(X))
p = 0
q = X.shape[0]*X.shape[1]*X.shape[2]
for A_ in A:
    p = p + A_.shape[0]*A_.shape[1]

print("Number of Parameters in the model: ", p)
print("Number of elements in the data: ", q)



The algorithm has converged. Number of iterations in ALS:  299
Norm of the Residual:  1.3315481783357157
Norm of the Data:  11.60200682748066
Number of Parameters in the model:  65
Number of elements in the data:  48


In [16]:
# Tucker3 Decomposition 

# Data
X = ar3.random_array3(tensor_shape)
rank_ = [3,3,3]

# Fit Tucker model
(A,G) = ar3.tucker3(X,rank_)
X_ = ar3.tucker_product(G,A,[1,2,3])

# Residual Analysis
print("Norm of the residual: ", np.linalg.norm(X-X_))
print("Norm of the tensor X: ",np.linalg.norm(X))

p = 0
for A_ in A:
    p = p + A_.shape[0]*A_.shape[1]
p = p + G.shape[0]*G.shape[1]*G.shape[2]
print("Number of elements in X: ",X.shape[0]*X.shape[1]*X.shape[2])
print("Number of Parameters: ", p)

The algorithm has converged. Number of iterations:  9
Norm of the residual:  1.0266013090758512
Norm of the tensor X:  19.347443669826557
Number of elements in X:  48
Number of Parameters:  55


In [19]:
# Array-Normal Distribution Test

# Generate Data
X = [ar3.random_array3(tensor_shape) for k in range(10)]
# Or use: X = [np.random.randn(*tensor_shape) for k in range(10)]

# Fit array-normal model
(M,Cov,A) = ar3.anormal(X)

# Sampling from the array-normal distribution 
Xs = ar3.anormal_sampling(M,Cov)
print(M.shape)
print([Cov[j].shape for j in range(3)])
# Array-normal Conditioning
Xt = X[0]
slice_ = 3
Ia = [0,1]
X_a = Xt[Ia,:,:]
#M_,Cov_ = ar3.anormal_condition(M,Cov,Ia,X_a, slice_)


MLE converged in  29  steps
(2, 8, 3)
[(8, 8), (3, 3), (2, 2)]


In [11]:
# Tensor-Tensor Regression Test

# Generate Data
X = ar3.random_array3(tensor_shape)
Y = X*2 + 0.1

# Fit a regression model and test the fit for different alpha (hyperparameter)
reps = 0
alpha = 0
while reps<3:
    reps = reps+1
    alpha=alpha+0.3
    print("For alpha %s :"%{alpha})
    R_X=[3,3,5]
    R_Y = R_X
    (Ax,Gx,Ay,Gy,W,Py)=ar3.cov_regression(X,Y, alpha, R_X, R_Y)
    y_mag = 0
    err = 0
    for k in range(tensor_shape[2]):
        X_test = X[k,:,:]
        a = mat_mul([W.T, X_test.flatten()])
        Y_test = mat_mul([a,Py])
        y_mag = y_mag + np.linalg.norm(Y_test)
        err = err + np.linalg.norm(Y_test-Y[k,:,:].flatten())

    print("Expected error in regression: ", err/tensor_shape[2])   
    print("Expected norm of Y: ", y_mag/tensor_shape[2])  


For alpha {0.3} :
Error in input data fit:  13.846750759099939 and norm of the input data:  63.38296447349815
Expected error in regression:  11.095367620031979
Expected norm of Y:  43.537658230899915
For alpha {0.6} :
Error in input data fit:  9.273264944292471 and norm of the input data:  63.38296447349815
Expected error in regression:  10.883239821742336
Expected norm of Y:  43.584117290660075
For alpha {0.8999999999999999} :
Error in input data fit:  12.643707193427314 and norm of the input data:  63.38296447349815
Expected error in regression:  11.859158140142206
Expected norm of Y:  43.48871826754525


In [ ]:
# For the following code you need tensorly package from http://tensorly.org/ 
import tensorly as tl
from tensorly.decomposition import tucker, parafac, non_negative_tucker
rank_ = [6,11,4]
(G_,A_) = tucker(X, ranks=rank_)
X_ = tl.tucker_to_tensor(G_,A_)
print(np.linalg.norm(X_-X))

A_ = parafac(X, rank=R)
Xa = mat_mul([A_[0],(ar3.khatri_rao([A_[1],A_[2]])).T])
X_ = tl.unfold(X,0)
print(np.linalg.norm(X_-Xa))